# LLNL WPS Client

This tutorial will demonstrate the features of the LLNL WPS Client.

**Features**
- LLNLAuthenticator handles authentication and token retrieval.
  - Can store token in between sessions.
- LLNLClient provides access to job management.

First we'll run a little setup then move onto looking at [authentication](#Authentication).

In [10]:
import os
import owslib
import cwt
from cwt import llnl_client

server_url = os.environ.get('SERVER_URL', 'https://aims2.llnl.gov/')
verify = bool(os.environ.get('VERIFY', True))
wps_url = os.environ.get('WPS_URL', 'https://aims2.llnl.gov/wps')

## Authentication

By default 3 authenticators are offered.

* [OWSLib authentication](#OWSLib-authentication)
* [LLNL authentication](#LLNL-authentication)
* [Token Authenticator](#Token-authentication) 

### OWSLib authentication

This method provides access to secure WPS servers requiring basic authentication or certificates.

In [3]:
auth = owslib.util.Authentication(username='test', password='test')
auth

<Authentication shared=False username=test password=test cert=None verify=True>

### LLNL authentication

This method utilizes the LLNL compute authentication which uses ESGF OpenID service.

In [4]:
auth = llnl_client.LLNLAuthenticator(server_url=server_url)
auth

LLNLAuthenticator(server_url='https://aims2.llnl.gov/', openid_url='https://esgf-node.llnl.gov/esgf-idp/openid', verify=True)

### Token authentication

This method uses and existing token to that will be passed in the header as `Authorization: Bearer <token>`.

In [6]:
auth = llnl_client.TokenAuthenticator('token')
auth

# Job history

The LLNL client provides access to job history and details.

In [8]:
auth = llnl_client.LLNLAuthenticator(server_url)

client = llnl_client.LLNLClient(wps_url, auth=auth)
client

WPSClient(url='https://aims2.llnl.gov/wps', log=False, log_file=None, verify=True, version=None, cert='1.0.0', headers={})

In [11]:
inputs = cwt.Variable('http://aims3.llnl.gov/thredds/dodsC/css03_data/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r10i1p1f1/Amon/tas/gn/v20180830/tas_Amon_GISS-E2-1-G_historical_r10i1p1f1_gn_190101-195012.nc', 'tas')

In [12]:
subset = client.CDAT.subset(inputs, domain=cwt.Domain(time=('1910', '1925'), lat=(0, 90)))
subset

Process(name=77c4d444, identifier=CDAT.subset, inputs=[Variable(name='c51f963d', uri='http://aims3.llnl.gov/thredds/dodsC/css03_data/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r10i1p1f1/Amon/tas/gn/v20180830/tas_Amon_GISS-E2-1-G_historical_r10i1p1f1_gn_190101-195012.nc', var_name='tas', domain=None, mime_type=None)], parameters={}, domain=Domain(Dimension(name='time', start='1910', end='1925', step=1, crs=CRS(name='timestamps')), Dimension(name='lat', start=0, end=90, step=1, crs=CRS(name='values')), mask=None, name='cfc5d2a6'), title=CDAT.subset, process_outputs=[], data_inputs=[], status_supported=None, store_supported=None, process_version=devel)

In [13]:
client.execute(subset)

subset.wait()

ProcessAccepted None
ProcessAccepted Validating inputs of CDAT.subset (f0ff3a53)
ProcessStarted Preparing inputs for process CDAT.subset (f0ff3a53) 0
ProcessStarted Building output for 'f0ff3a53' - 'CDAT.subset' 0
ProcessStarted Processing 0
ProcessSucceeded None


True

## You can list your job history

This defaults to 10 entries per page. You can navigate the pages using `jobs.previous()` and `jobs.next()` calls.

In [15]:
jobs = client.jobs()

# next page of jobs
jobs.next()

# previous page of jobs
jobs.previous()

jobs

ID,Operation,Elapsed,Status,Accepted
1,CDAT.workflow,40.839176,ProcessSucceeded,2020-04-27T23:50:06.507856+00:00
2,CDAT.workflow,4.16908,ProcessFailed,2020-04-28T06:10:40.100827+00:00
3,CDAT.workflow,0.602493,ProcessFailed,2020-04-28T06:28:48.948727+00:00
4,CDAT.workflow,0.702808,ProcessFailed,2020-04-28T06:43:11.890048+00:00
5,CDAT.workflow,171.103173,ProcessSucceeded,2020-04-28T06:47:55.650085+00:00
6,CDAT.workflow,70.106237,ProcessSucceeded,2020-04-28T07:15:44.621830+00:00
7,CDAT.workflow,No elapsed,ProcessFailed,2020-04-28T07:21:36.321473+00:00
8,CDAT.workflow,205.231695,ProcessFailed,2020-04-28T07:24:53.893896+00:00
9,CDAT.workflow,No elapsed,ProcessAccepted,2020-04-28T07:29:27.010552+00:00
10,CDAT.workflow,68.897268,ProcessSucceeded,2020-04-28T07:30:43.122024+00:00


## You can view the specifics of a job

Using the ID column from the job listing you can view the details of a job.

In [16]:
jobs[1]

Status,Created,Output
ProcessAccepted,2020-04-27T16:50:06.507856-07:00,2020-04-27T16:50:08.749669-07:00 Validating parameters of CDAT.merge (13ea28db) 0.0
,,2020-04-27T16:50:08.683771-07:00 Validating inputs of CDAT.merge (13ea28db) 0.0
,,2020-04-27T16:50:08.639958-07:00 Validating parameters of CDAT.divide (dcd22654) 0.0
,,2020-04-27T16:50:08.566276-07:00 Validating inputs of CDAT.divide (dcd22654) 0.0
,,2020-04-27T16:50:08.494757-07:00 Validating parameters of CDAT.divide (1d050e39) 0.0
,,2020-04-27T16:50:08.452903-07:00 Validating inputs of CDAT.divide (1d050e39) 0.0
,,2020-04-27T16:50:08.387025-07:00 Validating parameters of CDAT.divide (fae04839) 0.0
,,2020-04-27T16:50:08.345092-07:00 Validating inputs of CDAT.divide (fae04839) 0.0
,,2020-04-27T16:50:08.280597-07:00 Validating parameters of CDAT.divide (489265f2) 0.0
,,2020-04-27T16:50:08.237691-07:00 Validating inputs of CDAT.divide (489265f2) 0.0
